<a href="https://colab.research.google.com/github/Ihalagedara/FYP2-IDS/blob/main/test_loss_function_msle_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data analysis

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import mean_absolute_error
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
#import kerastuner as kt

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from tensorflow.keras.losses import MeanSquaredLogarithmicError

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/FYP IDS SDN/FYP ML/InSDN_DatasetCSV/new_dataset.csv', sep=",")
data_a = pd.read_csv('/content/drive/Shareddrives/FYP IDS SDN/FYP ML/InSDN_DatasetCSV/new_dataset.csv', sep=",")

In [ ]:
#condition3 = data['Label'] != 'Probe'
condition4 = data['Label'] != 'Normal'
condition3 = data['Label'] == 'Normal'
data = data.drop(data[condition4].index)
data_a = data_a.drop(data[condition3].index)

In [ ]:
#data['Label'] = data['Label'].replace('Probe', 1)
data['Label'] = data['Label'].replace('Normal', 0)

In [ ]:
data = data.drop(['Flow ID','Src IP','Src Port','Dst IP', 'Dst Port', 'Timestamp'], axis=1)
data_a = data_a.drop(['Flow ID','Src IP','Src Port','Dst IP', 'Dst Port', 'Timestamp'], axis=1)


In [ ]:
data_train, data_test = train_test_split(data, test_size=0.2)

In [ ]:
y_data = data_train["Label"]
data_train = data_train.drop(["Label"],axis=1)
y_data_test = data_test["Label"]
data_test = data_test.drop(["Label"],axis=1)
y_data_a = data_a["Label"]
data_a = data_a.drop(["Label"],axis=1)


In [ ]:
data = data.drop(["Label"],axis=1)

In [ ]:
#selected_features = ['Protocol', 'Tot Fwd Pkts', 'Flow Pkts/s', 'Fwd Header Len', 'Bwd Header Len', 'Bwd Pkts/s', 'Pkt Len Max', 'Init Bwd Win Byts']
#data = data.drop(columns=data.columns.difference(selected_features))
#data_test = data_test.drop(columns=data_test.columns.difference(selected_features))

In [ ]:
data = tf.constant(data)
data_train = tf.constant(data_train)
data_test = tf.constant(data_test)
data_a = tf.constant(data_a)

In [ ]:
'''mean = np.mean(data, axis=0)
std_dev = np.std(data, axis=0)

normalized_data = (data - mean) / std_dev'''

In [ ]:
scaler = StandardScaler()
data_scaled = scaler.fit(data)

In [ ]:
data = data_scaled.transform(data)
data_train = data_scaled.transform(data_train)
data_test = data_scaled.transform(data_test)
data_a = data_scaled.transform(data_a)

### model

In [ ]:
class AutoEncoder(tf.keras.models.Model):
  def __init__(self):
    super(AutoEncoder, self).__init__()
    self.encoder = tf.keras.Sequential([
        #tf.keras.layers.Dense(units=120, input_shape=(77,), activation='relu'),
        #tf.keras.layers.Embedding(input_dim=120, output_dim=66),
        tf.keras.layers.Dense(126, input_shape=(77,), activation='relu'),
        tf.keras.layers.Dense(460, activation='relu'),
        tf.keras.layers.Dense(680, activation='relu'),
        tf.keras.layers.Dense(148, activation='relu'),
        tf.keras.layers.Dense(534, activation='relu'),
        tf.keras.layers.Dense(762, activation='relu'),
        tf.keras.layers.Dense(536, activation='relu'),
        tf.keras.layers.Dense(228, activation='relu'),
        tf.keras.layers.Dense(334, activation='relu'),
        tf.keras.layers.Dense(58, activation='relu'),
    ])

    self.decoder = tf.keras.Sequential([
        tf.keras.layers.Dense(334, activation='relu'),
        tf.keras.layers.Dense(228, activation='relu'),
        tf.keras.layers.Dense(536, activation='relu'),
        tf.keras.layers.Dense(762, activation='relu'),
        tf.keras.layers.Dense(534, activation='relu'),
        tf.keras.layers.Dense(148, activation='relu'),
        tf.keras.layers.Dense(680, activation='relu'),
        tf.keras.layers.Dense(460, activation='linear'),
        tf.keras.layers.Dense(126, activation='linear'),
        tf.keras.layers.Dense(77, activation='linear'),
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [ ]:
model = AutoEncoder()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='mae',metrics=["accuracy"])

In [ ]:
dl = model.fit(data_train, data_train, epochs=20, batch_size=64,validation_split=0.2)

In [ ]:
#dl = best_model = tuner1.get_best_models(num_models=1)[0]

In [ ]:
print(f"max accuracy {max(dl.history['accuracy'])} : index {dl.history['accuracy'].index(max(dl.history['accuracy']))}")
print(f"max val accuracy {max(dl.history['val_accuracy'])} : index {dl.history['val_accuracy'].index(max(dl.history['val_accuracy']))}")
print(f"min loss {min(dl.history['loss'])} : index {dl.history['loss'].index(min(dl.history['loss']))}")
print(f"min val loss {min(dl.history['val_loss'])} : index {dl.history['val_loss'].index(min(dl.history['val_loss']))}")

In [ ]:
from matplotlib import pyplot as plt


In [ ]:
encoder_out = model.encoder(data_test).numpy()
decoder_out = model.decoder(encoder_out).numpy()

In [ ]:
reconstruction = model.predict(data_test)
train_loss = tf.keras.losses.mse(reconstruction,data_test)

In [ ]:
threshold = np.mean(train_loss) + 2*np.std(train_loss)

In [ ]:
threshold

In [ ]:
encoder_a = model.encoder(data_a).numpy()
decoder_a = model.decoder(encoder_out).numpy()

In [ ]:
reconstruction_a = model.predict(data_a)
train_loss_a = tf.keras.losses.msle(reconstruction_a,data_a)

### Testing threshold

In [ ]:
train_loss_msle = tf.keras.losses.msle(reconstruction,data_test)
train_loss_mse = tf.keras.losses.mse(reconstruction,data_test)
train_loss_mae = tf.keras.losses.mae(reconstruction,data_test)

In [ ]:
threshold_msle = np.mean(train_loss_msle) + 2*np.std(train_loss_msle)
threshold_mse = np.mean(train_loss_mse) + 2*np.std(train_loss_mse)
threshold_mae = np.mean(train_loss_mae) + 2*np.std(train_loss_mae)

In [ ]:
print(threshold_msle)
print(threshold_mse)
print(threshold_mae)

In [ ]:
print((tf.keras.losses.kld(reconstruction,data_test)))

## testing

In [ ]:
#loss
def loss(true_val,Pred_val):
  loss1 = tf.keras.losses.mean_squared_logarithmic_error(true_val, Pred_val)
  return loss1

### normal

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/FYP IDS SDN/FYP ML/InSDN_DatasetCSV/new_dataset.csv', sep=",")
data = data.drop(['Flow ID','Src IP','Src Port','Dst IP', 'Dst Port', 'Timestamp'], axis=1)
#selected_features = ['Protocol', 'Tot Fwd Pkts', 'Flow Pkts/s', 'Fwd Header Len', 'Bwd Header Len', 'Bwd Pkts/s', 'Pkt Len Max', 'Init Bwd Win Byts' , 'Label']
#data = data.drop(columns=data.columns.difference(selected_features))
condition4 = data['Label'] != 'Normal'
data = data.drop(data[condition4].index)
y_data_normal = data["Label"]
data_normal = data.drop(["Label"],axis=1)
data_normal = data_scaled.transform(data_normal)

In [ ]:
reconstructions_normal = model.predict(np.array(data_normal))
mse_normal = loss(data_normal , reconstructions_normal)

In [ ]:
#eig_normal = np.linalg.eigvals(np.dot(data_normal,data_normal.T))
#eig_normal

### bfa

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/FYP IDS SDN/FYP ML/InSDN_DatasetCSV/new_dataset.csv', sep=",")
data = data.drop(['Flow ID','Src IP','Src Port','Dst IP', 'Dst Port', 'Timestamp'], axis=1)
selected_features = ['Protocol', 'Tot Fwd Pkts', 'Flow Pkts/s', 'Fwd Header Len', 'Bwd Header Len', 'Bwd Pkts/s', 'Pkt Len Max', 'Init Bwd Win Byts' , 'Label']
#data = data.drop(columns=data.columns.difference(selected_features))
condition4 = data['Label'] != 'BFA'
data = data.drop(data[condition4].index)
y_data_bfa = data["Label"]
data_bfa = data.drop(["Label"],axis=1)
data_bfa = data_scaled.transform(data_bfa)

In [ ]:
reconstructions_bfa = model.predict(np.array(data_bfa))
mse_bfa = loss(data_bfa , reconstructions_bfa)

### ddos

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/FYP IDS SDN/FYP ML/InSDN_DatasetCSV/new_dataset.csv', sep=",")
data = data.drop(['Flow ID','Src IP','Src Port','Dst IP', 'Dst Port', 'Timestamp'], axis=1)
selected_features = ['Protocol', 'Tot Fwd Pkts', 'Flow Pkts/s', 'Fwd Header Len', 'Bwd Header Len', 'Bwd Pkts/s', 'Pkt Len Max', 'Init Bwd Win Byts' , 'Label']
#data = data.drop(columns=data.columns.difference(selected_features))
condition4 = data['Label'] != 'DDoS '
condition1 = data['Label'] != 'DDoS'
data = data.drop(data[condition4 & condition1].index)
y_data_ddos = data["Label"]
data_ddos = data.drop(["Label"],axis=1)
data_ddos = data_scaled.transform(data_ddos)

In [ ]:
reconstructions_ddos = model.predict(np.array(data_ddos))
mse_ddos = loss(data_ddos , reconstructions_ddos)


### dos

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/FYP IDS SDN/FYP ML/InSDN_DatasetCSV/new_dataset.csv', sep=",")
data = data.drop(['Flow ID','Src IP','Src Port','Dst IP', 'Dst Port', 'Timestamp'], axis=1)
selected_features = ['Protocol', 'Tot Fwd Pkts', 'Flow Pkts/s', 'Fwd Header Len', 'Bwd Header Len', 'Bwd Pkts/s', 'Pkt Len Max', 'Init Bwd Win Byts' , 'Label']
#data = data.drop(columns=data.columns.difference(selected_features))
condition4 = data['Label'] != 'DoS'
data = data.drop(data[condition4].index)
y_data_dos = data["Label"]
data_dos = data.drop(["Label"],axis=1)
data_dos = data_scaled.transform(data_ddos)

In [ ]:
reconstructions_dos = model.predict(np.array(data_dos))
mse_dos = loss(data_dos ,reconstructions_dos)


### Probe

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/FYP IDS SDN/FYP ML/InSDN_DatasetCSV/new_dataset.csv', sep=",")
data = data.drop(['Flow ID','Src IP','Src Port','Dst IP', 'Dst Port', 'Timestamp'], axis=1)
selected_features = ['Protocol', 'Tot Fwd Pkts', 'Flow Pkts/s', 'Fwd Header Len', 'Bwd Header Len', 'Bwd Pkts/s', 'Pkt Len Max', 'Init Bwd Win Byts' , 'Label']
#data = data.drop(columns=data.columns.difference(selected_features))
condition4 = data['Label'] != 'Probe'
data = data.drop(data[condition4].index)
y_data_prob = data["Label"]
data_prob = data.drop(["Label"],axis=1)
data_prob = data_scaled.transform(data_prob)

In [ ]:
reconstructions_prob = model.predict(np.array(data_prob))
mse_prob = loss(data_prob , reconstructions_prob)

### Web-Attack

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/FYP IDS SDN/FYP ML/InSDN_DatasetCSV/new_dataset.csv', sep=",")
data = data.drop(['Flow ID','Src IP','Src Port','Dst IP', 'Dst Port', 'Timestamp'], axis=1)
selected_features = ['Protocol', 'Tot Fwd Pkts', 'Flow Pkts/s', 'Fwd Header Len', 'Bwd Header Len', 'Bwd Pkts/s', 'Pkt Len Max', 'Init Bwd Win Byts' , 'Label']
#data = data.drop(columns=data.columns.difference(selected_features))
condition4 = data['Label'] != 'Web-Attack'
data = data.drop(data[condition4].index)
y_data_WebAttack = data["Label"]
data_WebAttack = data.drop(["Label"],axis=1)
data_WebAttack = data_scaled.transform(data_WebAttack)

In [ ]:
reconstructions_WebAttack = model.predict(np.array(data_WebAttack))
mse_WebAttack = loss(data_WebAttack , reconstructions_WebAttack)

### BOTNET

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/FYP IDS SDN/FYP ML/InSDN_DatasetCSV/new_dataset.csv', sep=",")
data = data.drop(['Flow ID','Src IP','Src Port','Dst IP', 'Dst Port', 'Timestamp'], axis=1)
selected_features = ['Protocol', 'Tot Fwd Pkts', 'Flow Pkts/s', 'Fwd Header Len', 'Bwd Header Len', 'Bwd Pkts/s', 'Pkt Len Max', 'Init Bwd Win Byts' , 'Label']
#data = data.drop(columns=data.columns.difference(selected_features))
condition4 = data['Label'] != 'BOTNET'
data = data.drop(data[condition4].index)
y_data_BOTNET = data["Label"]
data_BOTNET = data.drop(["Label"],axis=1)
data_BOTNET = data_scaled.transform(data_BOTNET)

In [ ]:
reconstructions_BOTNET = model.predict(np.array(data_BOTNET))
mse_BOTNET = loss(data_BOTNET , reconstructions_BOTNET)

### U2R

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/FYP IDS SDN/FYP ML/InSDN_DatasetCSV/new_dataset.csv', sep=",")
data = data.drop(['Flow ID','Src IP','Src Port','Dst IP', 'Dst Port', 'Timestamp'], axis=1)
selected_features = ['Protocol', 'Tot Fwd Pkts', 'Flow Pkts/s', 'Fwd Header Len', 'Bwd Header Len', 'Bwd Pkts/s', 'Pkt Len Max', 'Init Bwd Win Byts' , 'Label']
#data = data.drop(columns=data.columns.difference(selected_features))
condition4 = data['Label'] != 'U2R'
data = data.drop(data[condition4].index)
y_data_U2R = data["Label"]
data_U2R = data.drop(["Label"],axis=1)
data_U2R = data_scaled.transform(data_U2R)

In [ ]:
reconstructions_U2R = model.predict(np.array(data_U2R))
mse_U2R = loss(data_U2R , reconstructions_U2R)

In [ ]:
#threshold = np.percentile(mse, 95)

#is_anomaly = mse < threshold

### out

In [ ]:
print(f"normal \t max {np.max(mse_normal)} \t min {np.min(mse_normal)}")
print(f"mse_bfa \t max {np.max(mse_bfa)} \t min {np.min(mse_bfa)}")
print(f"mse_ddos \t max {np.max(mse_ddos)} \t min {np.min(mse_ddos)}")
print(f"mse_prob \t max {np.max(mse_prob)} \t min {np.min(mse_prob)}")
print(f"mse_WebAttack \t max {np.max(mse_WebAttack)} \t min {np.min(mse_WebAttack)}")
print(f"mse_BOTNET \t max {np.max(mse_BOTNET)} \t min {np.min(mse_BOTNET)}")
print(f"mse_U2R \t max {np.max(mse_U2R)} \t min {np.min(mse_U2R)}")
print(f"mse_dos \t max {np.max(mse_dos)} \t min {np.min(mse_dos)}")

In [ ]:
threshold_normal = np.percentile(mse_normal, 85)
threshold_bfa = np.percentile(mse_bfa, 95)
threshold_ddos = np.percentile(mse_ddos, 95)
threshold_prob = np.percentile(mse_prob, 95)
threshold_WebAttack = np.percentile(mse_WebAttack, 95)
threshold_BOTNET = np.percentile(mse_BOTNET, 95)
threshold_U2R = np.percentile(mse_U2R, 95)

In [ ]:
print(threshold_normal)
print(threshold_bfa)
print(threshold_ddos)
print(threshold_prob)
print(threshold_WebAttack)
print(threshold_BOTNET)
print(threshold_U2R)

## accuracy

In [ ]:
threshold_mae = np.percentile(mse_normal, 85)

In [ ]:
threshold_mae

In [ ]:
anom_normal = mse_normal < threshold_mae
anom_bfa = mse_bfa > threshold_mae
anom_ddos  = mse_ddos > threshold_mae
anom_prob = mse_prob > threshold_mae
anom_WebAttack = mse_WebAttack > threshold_mae
anom_BOTNET = mse_BOTNET >  threshold_mae
anom_U2R = mse_U2R > threshold_mae
anom_dos = mse_dos > threshold_mae

In [ ]:
def check(anom):
  count =0
  var_name = [name for name, value in globals().items() if value is anom][0]
  for i in range(len(anom)):
    if np.array(anom)[i]:
      count = count +1

  print(f"{var_name} \t accuracy: {count/len(anom)*100:.3f} \t length: {len(anom)}")

In [ ]:
check(anom_normal)
check(anom_bfa)
check(anom_ddos)
check(anom_prob)
check(anom_WebAttack)
check(anom_BOTNET)
check(anom_U2R)
check(anom_dos)